In [1]:
"""Tests MBR_X built on alphabet on 3 tasks
1. PSSM information content
2. duoalignment for family retrieval
3. pairwise alignment with beagle
"""

'Tests MBR_X built on alphabet on 3 tasks\n1. PSSM information content\n2. duoalignment for family retrieval\n3. pairwise alignment with beagle\n'

In [14]:
#imports and testing variables
import sys
import argparse
import pickle
import pandas as pd
import numpy as np

from matrix2 import *

mbrVersion = "qbear_50"

MBR = "MBR_qbear_50.tsv"
ALPHAMAP = "qBEAR.txt"
GAPFAMDICT = "gapped_fam_dict.pickle"

ignore_these_families = ['RF00210','RF01879', 'RF02767', 'RF02768','RF02770', 'RF02773', 'RF02775','RF02781', 'RF02783']


In [3]:
#adding parser
parser = argparse.ArgumentParser()
parser.add_argument("-m", "--mbr", help="the substitution matrix to test")
parser.add_argument("-a", "--alpha", help="pickle of alphabet dictionary (must correspond to the alphabet \
                                            used with the substitution matrix). With respect to standard BEAR")
parser.add_argument("-gfd", "--gapfamdict", help="the rfam alignments with gaps and bear pickle")
parser.add_argument("-v", "--verbose", help="increase output verbosity",
                    action="store_true")

args = parser.parse_args(args=['--alpha', ALPHAMAP,
                               '-v', 
                               '-m',MBR,
                              '-gfd', GAPFAMDICT])

In [4]:
#####VERSIONE PRECEDENTE CON ALFABETO IN DIZIONARIO#####
'''
###Variable assignment
with open(args.gapfamdict, 'rb') as afile:
    gapped_fam_dict = pickle.load(afile)

with open(args.alpha, 'rb') as afile:
    alph_bear = pickle.load(afile)

alphamaps = [{key:alph_bear[KEY] for key in KEY}
            for KEY in alph_bear]
alphamap = {}
for ap in alphamaps:
    alphamap.update(ap)
    
with open(args.mbr, 'rb') as afile:
    mbr = pickle.load(afile, encoding='latin1')
    ###ATTENZIONE, ENCODING serve solo se il pickle arriva da python 2.7 (marta), altrimenti non so come va
'''

"\n###Variable assignment\nwith open(args.gapfamdict, 'rb') as afile:\n    gapped_fam_dict = pickle.load(afile)\n\nwith open(args.alpha, 'rb') as afile:\n    alph_bear = pickle.load(afile)\n\nalphamaps = [{key:alph_bear[KEY] for key in KEY}\n            for KEY in alph_bear]\nalphamap = {}\nfor ap in alphamaps:\n    alphamap.update(ap)\n    \nwith open(args.mbr, 'rb') as afile:\n    mbr = pickle.load(afile, encoding='latin1')\n    ###ATTENZIONE, ENCODING serve solo se il pickle arriva da python 2.7 (marta), altrimenti non so come va\n"

In [5]:
#####VERSIONE NUOVA CON ALFABETO IN FILE TSV#####

###Variable assignment
with open(args.gapfamdict, 'rb') as afile:
    gapped_fam_dict = pickle.load(afile)
    
    
f=open(args.alpha).readlines()
alph_bear={x.split()[0]:x.split()[1] for x in f}
alph_bear['-']='-'

alphamaps = [{key:alph_bear[KEY] for key in KEY} for KEY in alph_bear]
alphamap = {}
for ap in alphamaps:
    alphamap.update(ap)
    
mbr=pd.read_table(args.mbr, index_col=0)

In [6]:
mbr
#scratchbook


,z,a,q,x,s,w,c,d,e,b,g,t,v,f,r,n,h,y
z,1.251901,-1.004386,-1.875156,-1.924021,-1.246907,-0.090560,-0.969852,0.118495,0.197494,-0.500413,-0.544214,-1.632660,-0.753703,-2.837002,-3.680622,-0.799709,-0.396543,-0.544076
a,-1.004386,2.183718,0.083500,-1.622771,-1.508918,-0.751584,-1.505466,-0.766710,-0.744210,-0.621187,-0.415789,-2.134194,-1.088041,-1.022524,-3.212613,-0.786086,-0.635661,-0.303277
q,-1.875156,0.083500,4.938520,-2.839489,-2.938266,-2.213520,-1.889927,-1.875586,-1.631255,-0.064752,-1.390353,-2.819128,-2.167109,-2.927971,-0.885244,-1.651218,-2.198932,-3.420976
x,-1.924021,-1.622771,-2.839489,2.601893,0.329162,0.322428,-1.233105,-0.332829,-0.416625,-0.850567,-1.044173,-2.227982,-1.734505,-3.548809,-3.728150,0.029378,-0.393977,-0.360536
s,-1.246907,-1.508918,-2.938266,0.329162,5.591254,2.523561,-0.659044,0.136920,0.158087,-0.518327,-1.022181,-1.674776,-1.751805,-3.417864,-3.926585,-0.115749,0.866039,1.174097
w,-0.090560,-0.751584,-2.213520,0.322428,2.523561,8.110535,0.911527,1.478493,2.926003,0.192768,-1.335073,-0.063744,-1.382303,-3.214545,-4.061672,-0.302880,0.538770,2.642098
c,-0.969852,-1.505466,-1.889927,-1.233105,-0.659044,0.911527,3.514573,1.112111,1.307975,0.580190,-0.434166,-1.145477,-1.300833,-3.187975,-3.734170,0.416329,0.318929,-0.005568
d,0.118495,-0.766710,-1.875586,-0.332829,0.136920,1.478493,1.112111,5.469184,2.311892,0.331947,-0.224466,-0.058633,-0.613905,-2.884740,-3.340735,0.177721,1.034403,1.310405
e,0.197494,-0.744210,-1.631255,-0.416625,0.158087,2.926003,1.307975,2.311892,5.522222,0.590471,0.250965,0.289942,-0.087344,-2.389063,-3.928441,1.077903,1.937196,1.871875
b,-0.500413,-0.621187,-0.064752,-0.850567,-0.518327,0.192768,0.580190,0.331947,0.590471,4.791997,0.483115,-0.846769,-0.640583,-2.365144,-3.288815,-0.344893,-0.461127,-0.753164


In [7]:
alphamap

{'!': 'c',
 '"': 'c',
 '#': 'c',
 '$': 'c',
 '%': 'c',
 '&': 'd',
 "'": 'd',
 '(': 'd',
 ')': 'd',
 '+': 'e',
 '-': '-',
 '/': 'h',
 '0': 'd',
 '2': 'c',
 '3': 'c',
 '4': 'c',
 '5': 'c',
 '6': 'c',
 '7': 'd',
 '8': 'd',
 '9': 'd',
 ':': 't',
 '=': 'q',
 '>': 'e',
 '?': 'n',
 '@': 'y',
 'A': 'v',
 'B': 'v',
 'C': 'v',
 'D': 'v',
 'E': 'v',
 'F': 'f',
 'G': 'f',
 'H': 'f',
 'I': 'f',
 'J': 'r',
 'K': 'n',
 'L': 'n',
 'M': 'n',
 'N': 'n',
 'O': 'h',
 'P': 'h',
 'Q': 'h',
 'R': 'h',
 'S': 'h',
 'T': 'y',
 'U': 'y',
 'V': 'y',
 'W': 'y',
 'Y': 'n',
 'Z': 'n',
 '[': 'b',
 '\\': 'h',
 ']': 'b',
 '^': 'w',
 '_': 'h',
 'a': 'z',
 'b': 'z',
 'c': 'z',
 'd': 'z',
 'e': 'z',
 'f': 'a',
 'g': 'a',
 'h': 'a',
 'i': 'a',
 'l': 'x',
 'm': 'x',
 'n': 'x',
 'o': 'x',
 'p': 'x',
 'q': 'x',
 'r': 'x',
 's': 's',
 't': 's',
 'u': 's',
 'v': 's',
 'w': 's',
 'x': 's',
 'y': 's',
 'z': 's',
 '{': 'g',
 '|': 'h',
 '}': 'g',
 '~': 'n'}

In [8]:
def mapAligns(gapped_fam_dict, alphamap):
    """convert alignments in the new alphabet"""
    for RF in gapped_fam_dict:
        print(RF)
        for ID in gapped_fam_dict[RF]:
            if gapped_fam_dict[RF][ID].get('bear') :
                gapped_fam_dict[RF][ID]['alpha'] = "".join([alphamap[ch] 
                                                            for ch in gapped_fam_dict.get(RF).get(ID).get('bear')])



In [9]:
#qualcosa come la 210 non hanno struttura attenzione sono ancora dentro
mapAligns(gapped_fam_dict, alphamap)

RF00001
RF00002
RF00003
RF00004
RF00005
RF00006
RF00007
RF00008
RF00009
RF00010
RF00011
RF00012
RF00013
RF00014
RF00015
RF00016
RF00017
RF00018
RF00019
RF00020
RF00021
RF00022
RF00023
RF00024
RF00025
RF00026
RF00027
RF00028
RF00029
RF00030
RF00031
RF00032
RF00033
RF00034
RF00035
RF00036
RF00037
RF00038
RF00039
RF00040
RF00041
RF00042
RF00043
RF00044
RF00045
RF00046
RF00047
RF00048
RF00049
RF00050
RF00051
RF00052
RF00053
RF00054
RF00055
RF00056
RF00057
RF00058
RF00059
RF00060
RF00061
RF00062
RF00063
RF00064
RF00065
RF00066
RF00067
RF00068
RF00069
RF00070
RF00071
RF00072
RF00073
RF00074
RF00075
RF00076
RF00077
RF00078
RF00079
RF00080
RF00081
RF00082
RF00083
RF00084
RF00085
RF00086
RF00087
RF00088
RF00089
RF00090
RF00091
RF00092
RF00093
RF00094
RF00095
RF00096
RF00097
RF00099
RF00100
RF00101
RF00102
RF00103
RF00104
RF00105
RF00106
RF00107
RF00108
RF00109
RF00110
RF00111
RF00112
RF00113
RF00114
RF00115
RF00116
RF00117
RF00118
RF00119
RF00121
RF00122
RF00124
RF00125
RF00126
RF00127
RF00128


In [10]:
##transform data into { RF: [[sequences,...], [ bear, ...]]} (like mirnas)
## Difference : GAPS
rfams = {}
for RF in gapped_fam_dict:
    print(RF)
    if RF not in ignore_these_families:
        rfams[RF] = [ [gapped_fam_dict[RF][ID]['sequence'] for ID in gapped_fam_dict[RF]],
                     [gapped_fam_dict[RF][ID]['alpha'] for ID in gapped_fam_dict[RF]] ]
    

#rfams['RF00360']

RF00001
RF00002
RF00003
RF00004
RF00005
RF00006
RF00007
RF00008
RF00009
RF00010
RF00011
RF00012
RF00013
RF00014
RF00015
RF00016
RF00017
RF00018
RF00019
RF00020
RF00021
RF00022
RF00023
RF00024
RF00025
RF00026
RF00027
RF00028
RF00029
RF00030
RF00031
RF00032
RF00033
RF00034
RF00035
RF00036
RF00037
RF00038
RF00039
RF00040
RF00041
RF00042
RF00043
RF00044
RF00045
RF00046
RF00047
RF00048
RF00049
RF00050
RF00051
RF00052
RF00053
RF00054
RF00055
RF00056
RF00057
RF00058
RF00059
RF00060
RF00061
RF00062
RF00063
RF00064
RF00065
RF00066
RF00067
RF00068
RF00069
RF00070
RF00071
RF00072
RF00073
RF00074
RF00075
RF00076
RF00077
RF00078
RF00079
RF00080
RF00081
RF00082
RF00083
RF00084
RF00085
RF00086
RF00087
RF00088
RF00089
RF00090
RF00091
RF00092
RF00093
RF00094
RF00095
RF00096
RF00097
RF00099
RF00100
RF00101
RF00102
RF00103
RF00104
RF00105
RF00106
RF00107
RF00108
RF00109
RF00110
RF00111
RF00112
RF00113
RF00114
RF00115
RF00116
RF00117
RF00118
RF00119
RF00121
RF00122
RF00124
RF00125
RF00126
RF00127
RF00128


In [11]:
#scratchbook


In [12]:
%%time
PSSMs_alpha = []

rfam_list = []
#use RFAMS 
counter = 0
for rfam in rfams:
    print(rfam)
    rfam_list.append(rfam)  
    PSSM_b = buildPSSM_alphabet(rfams[rfam], mbr) ###TODO load MBR (dataframe)
    
    PSSMs_alpha.append(PSSM_b)
    counter+=1

RF00001
RF00002
RF00003
RF00004
RF00005
RF00006
RF00007
RF00008
RF00009
RF00010
RF00011
RF00012
RF00013
RF00014
RF00015
RF00016
RF00017
RF00018
RF00019
RF00020
RF00021
RF00022
RF00023
RF00024
RF00025
RF00026
RF00027
RF00028
RF00029
RF00030
RF00031
RF00032
RF00033
RF00034
RF00035
RF00036
RF00037
RF00038
RF00039
RF00040
RF00041
RF00042
RF00043
RF00044
RF00045
RF00046
RF00047
RF00048
RF00049
RF00050
RF00051
RF00052
RF00053
RF00054
RF00055
RF00056
RF00057
RF00058
RF00059
RF00060
RF00061
RF00062
RF00063
RF00064
RF00065
RF00066
RF00067
RF00068
RF00069
RF00070
RF00071
RF00072
RF00073
RF00074
RF00075
RF00076
RF00077
RF00078
RF00079
RF00080
RF00081
RF00082
RF00083
RF00084
RF00085
RF00086
RF00087
RF00088
RF00089
RF00090
RF00091
RF00092
RF00093
RF00094
RF00095
RF00096
RF00097
RF00099
RF00100
RF00101
RF00102
RF00103
RF00104
RF00105
RF00106
RF00107
RF00108
RF00109
RF00110
RF00111
RF00112
RF00113
RF00114
RF00115
RF00116
RF00117
RF00118
RF00119
RF00121
RF00122
RF00124
RF00125
RF00126
RF00127
RF00128


In [15]:
with open('rfam_PSSM_{}.pickle'.format(mbrVersion), 'wb') as handle:
    pickle.dump(PSSMs_alpha, handle)   

# STOP HERE, divide code in modules


# the code here can be executed with previously saved PSSMs (change testname if needed)

In [16]:
##
if "PSSMs_alpha" not in locals():
    with open('rfam_PSSM_{}.pickle'.format(mbrVersion), 'rb') as afile:
        PSSMs_alpha = pickle.load(afile)


In [17]:
"""1. PSSM information content
da Henikoff^2 pare sia facile calcolarlo on the go durante la costruzione delle matrici (marta?) 
"""
print(rfam_list)

['RF00001', 'RF00002', 'RF00003', 'RF00004', 'RF00005', 'RF00006', 'RF00007', 'RF00008', 'RF00009', 'RF00010', 'RF00011', 'RF00012', 'RF00013', 'RF00014', 'RF00015', 'RF00016', 'RF00017', 'RF00018', 'RF00019', 'RF00020', 'RF00021', 'RF00022', 'RF00023', 'RF00024', 'RF00025', 'RF00026', 'RF00027', 'RF00028', 'RF00029', 'RF00030', 'RF00031', 'RF00032', 'RF00033', 'RF00034', 'RF00035', 'RF00036', 'RF00037', 'RF00038', 'RF00039', 'RF00040', 'RF00041', 'RF00042', 'RF00043', 'RF00044', 'RF00045', 'RF00046', 'RF00047', 'RF00048', 'RF00049', 'RF00050', 'RF00051', 'RF00052', 'RF00053', 'RF00054', 'RF00055', 'RF00056', 'RF00057', 'RF00058', 'RF00059', 'RF00060', 'RF00061', 'RF00062', 'RF00063', 'RF00064', 'RF00065', 'RF00066', 'RF00067', 'RF00068', 'RF00069', 'RF00070', 'RF00071', 'RF00072', 'RF00073', 'RF00074', 'RF00075', 'RF00076', 'RF00077', 'RF00078', 'RF00079', 'RF00080', 'RF00081', 'RF00082', 'RF00083', 'RF00084', 'RF00085', 'RF00086', 'RF00087', 'RF00088', 'RF00089', 'RF00090', 'RF00091'

In [18]:
dic_PSSM={}
for i in range(0, len(rfam_list)):
    dic_PSSM[rfam_list[i]]=PSSMs_alpha[i]
    
print(dic_PSSM['RF00001'])

[[-1.99948407 -2.00117556 -2.30636696 ... -2.38616215 -2.27736274
  -1.68324488]
 [-2.00018847 -1.99862714 -1.42505492 ... -1.26604919 -1.41075805
  -2.08958232]
 [-2.00115046 -2.00130333 -2.71457754 ... -2.73724437 -2.73327748
  -2.65478745]
 ...
 [-1.9963381  -1.99874554 -0.62653502 ... -0.72081822 -0.55136007
   0.11941045]
 [-1.99611382 -1.9989964  -0.66787569 ... -0.80057883 -0.59104999
   0.24550539]
 [-2.         -2.         -2.         ... -2.         -2.
  -2.        ]]


In [20]:
PSSMs_alpha[0]

with open('rfam_PSSM_dic_{}.pickle'.format(mbrVersion), 'wb') as handle:
    pickle.dump(dic_PSSM, handle) 